<a href="https://colab.research.google.com/github/ny-yo/kaggle-CommonLit-Readability-Prize/blob/main/cat_dog_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#犬と猫を分類するマルチラベル分類のモデルを作成

AI Academyの下記を参照した  
https://aiacademy.jp/texts/show/?id=164

#google driveのマウント

In [1]:
from google.colab import drive #インポート
drive.mount('/content/drive/') #GoogleDriveのマウント

Mounted at /content/drive/


#画像収集ライブラリicawlerで画像データをダウンロード  
bingだと動作したがgoogleimageだとダウンロードできず

In [2]:
#pythonの画像収集ライブラリ
!pip install icrawler

  Created wheel for icrawler: filename=icrawler-0.6.4-py2.py3-none-any.whl size=35063 sha256=9d31382ae3e74bfaf845d9d6112f05d7685b07f6173489fea24552bc49172481
  Stored in directory: /root/.cache/pip/wheels/34/9a/e6/a8bc00ac61ff1f0e613af0d181feaef49ed226915ee7ff19d1
Successfully built icrawler


In [3]:
from icrawler.builtin import BingImageCrawler

# 猫の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "drive/MyDrive/KK_proto/images/cat"})
crawler.crawl(keyword="猫", max_num=100)

2021-08-07 10:16:26,471 - INFO - icrawler.crawler - start crawling...
2021-08-07 10:16:26,472 - INFO - icrawler.crawler - starting 1 feeder threads...
2021-08-07 10:16:26,474 - INFO - feeder - thread feeder-001 exit
2021-08-07 10:16:26,476 - INFO - icrawler.crawler - starting 1 parser threads...
2021-08-07 10:16:26,479 - INFO - icrawler.crawler - starting 1 downloader threads...
2021-08-07 10:16:26,776 - INFO - parser - parsing result page https://www.bing.com/images/async?q=猫&first=0
2021-08-07 10:16:27,478 - INFO - downloader - skip downloading file 000001.jpg
2021-08-07 10:16:27,480 - INFO - downloader - skip downloading file 000002.jpg
2021-08-07 10:16:27,484 - INFO - downloader - skip downloading file 000003.jpg
2021-08-07 10:16:27,486 - INFO - downloader - skip downloading file 000004.jpg
2021-08-07 10:16:27,488 - INFO - downloader - skip downloading file 000005.jpg
2021-08-07 10:16:27,490 - INFO - downloader - skip downloading file 000006.jpg
2021-08-07 10:16:27,491 - INFO - dow

In [4]:
from icrawler.builtin import BingImageCrawler

# 犬の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "drive/MyDrive/KK_proto/images/dog"})
crawler.crawl(keyword="犬", max_num=100)

2021-08-07 10:17:13,551 - INFO - icrawler.crawler - start crawling...
2021-08-07 10:17:13,551 - INFO - icrawler.crawler - starting 1 feeder threads...
2021-08-07 10:17:13,553 - INFO - feeder - thread feeder-001 exit
2021-08-07 10:17:13,554 - INFO - icrawler.crawler - starting 1 parser threads...
2021-08-07 10:17:13,559 - INFO - icrawler.crawler - starting 1 downloader threads...
2021-08-07 10:17:13,860 - INFO - parser - parsing result page https://www.bing.com/images/async?q=犬&first=0
2021-08-07 10:17:13,876 - INFO - downloader - skip downloading file 000001.jpg
2021-08-07 10:17:13,878 - INFO - downloader - skip downloading file 000002.jpg
2021-08-07 10:17:13,879 - INFO - downloader - skip downloading file 000003.jpg
2021-08-07 10:17:13,882 - INFO - downloader - skip downloading file 000004.jpg
2021-08-07 10:17:13,884 - INFO - downloader - skip downloading file 000005.jpg
2021-08-07 10:17:13,885 - INFO - downloader - skip downloading file 000006.jpg
2021-08-07 10:17:13,889 - INFO - dow

#データの整形と学習データの作成

In [5]:
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile
# IOError: image file is truncated (0 bytes not processed)回避のため
ImageFile.LOAD_TRUNCATED_IMAGES = True

classes = ["dog", "cat"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "drive/MyDrive/KK_proto/images/" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:

            # angleに代入される値
            # -20
            # -15
            # -10
            #  -5
            # 0
            # 5
            # 10
            # 15
            # 画像を5度ずつ回転
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                # FLIP_LEFT_RIGHT　は 左右反転
                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trains)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./dog_cat.npy", xy)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


#学習フェーズ

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras.utils import np_utils
import keras
import numpy as np

classes = ["dog", "cat"]
num_classes = len(classes)
image_size = 64

"""
データを読み込む関数
"""
def load_data():
    X_train, X_test, y_train, y_test = np.load("./dog_cat.npy", allow_pickle=True)
    # 入力データの各画素値を0-1の範囲で正規化(学習コストを下げるため)
    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255
    # to_categorical()にてラベルをone hot vector化
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test

"""
モデルを学習する関数
"""
def train(X, y, X_test, y_test):
    model = Sequential()

    # Xは(1200, 64, 64, 3)
    # X.shape[1:]とすることで、(64, 64, 3)となり、入力にすることが可能です。
    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    # https://keras.io/ja/optimizers/
    # 今回は、最適化アルゴリズムにRMSpropを利用
    opt = RMSprop(lr=0.00005, decay=1e-6)
    # https://keras.io/ja/models/sequential/
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)
    # HDF5ファイルにKerasのモデルを保存
    model.save('./cnn.h5')

    return model

"""
メイン関数
データの読み込みとモデルの学習を行います。
"""
def main():
    # データの読み込み
    X_train, y_train, X_test, y_test = load_data()

    # モデルの学習
    model = train(X_train, y_train, X_test, y_test)

main()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/40
74/74 [==============================] - 29s 9ms/step - loss: 0.6687 - accuracy: 0.5752
Epoch 2/40
74/74 [==============================] - 1s 8ms/step - loss: 0.6185 - accuracy: 0.6736
Epoch 3/40
74/74 [==============================] - 1s 8ms/step - loss: 0.5440 - accuracy: 0.7213
Epoch 4/40
74/74 [==============================] - 1s 8ms/step - loss: 0.4906 - accuracy: 0.7592
Epoch 5/40
74/74 [==============================] - 1s 8ms/step - loss: 0.4407 - accuracy: 0.8081
Epoch 6/40
74/74 [==============================] - 1s 8ms/step - loss: 0.4103 - accuracy: 0.8317
Epoch 7/40
74/74 [==============================] - 1s 8ms/step - loss: 0.3601 - accuracy: 0.8553
Epoch 8/40
74/74 [==============================] - 1s 7ms/step - loss: 0.3222 - accuracy: 0.8875
Epoch 9/40
74/74 [==============================] - 1s 8ms/step - loss: 0.2784 - accuracy: 0.9101
Epoch 10/40
74/74 [==============================] - 1s 8ms/step - loss: 0.2418 - accuracy: 0.9260
Epoch 11/40
74/74 

#推論モデル
テスト画像が犬or猫のどちらかを推論するモデル

In [7]:
import keras
import sys, os
import numpy as np
from keras.models import load_model

imsize = (64, 64)

"""
dog1.jpgというファイル名の画像をGoogle Colab上にアップロードする方法は2通りあります。
1つが、下記のコードを実行し画像をアップロードする方法
from google.colab import files
uploaded = files.upload()
2つが、Colab左メニューの>アイコンを押して、目次、コード スニペット、ファイル
の3つ表示されるますが、右のファイルタブから画像をアップロードする方法です。
このファイルタブをクリックするとアップロードと更新の2つがありますが、
アップロードを押すと画像をアップロードすることが可能です。
"""

testpic     = "./dog1.jpg"
keras_param = "./cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')
    # 学習時に、(64, 64, 3)で学習したので、画像の縦・横は今回 変数imsizeの(64, 64)にリサイズします。
    img = img.resize(imsize)
    # 画像データをnumpy配列の形式に変更
    img = np.asarray(img)
    img = img / 255.0
    return img

from google.colab import files
uploaded = files.upload()

model = load_model(keras_param)
img = load_image(testpic)
prd = model.predict(np.array([img]))
print(prd) # 精度の表示
prelabel = np.argmax(prd, axis=1)
if prelabel == 0:
    print(">>> 犬")
elif prelabel == 1:
    print(">>> 猫")

Saving dog1.jpg to dog1.jpg
[[9.995609e-01 4.391366e-04]]
>>> 犬
